In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
nx, ny = 32, 32 # number of computational grids
dx, dy = 0.5e-6, 0.5e-6 # spacing of computational grid [m]
eee = 1.0e+6 # driving force of growth of phase B: g_A - g_B [J/m3]
gamma = 1.0 # ineterfacial energy [J/m2]
delta = 4.*dx # interfacial thickness [m]
amobi = 4.e-14 # interfacial mobilitiy [m4/(Js)]
ram = 0.1 # paraneter which deternines the interfacial area
bbb = 2.*np.log((1.+(1.-2.*ram))/(1.-(1.-2.*ram)))/2.  # The constant b = 2.1972

In [ ]:
aaa   = np.sqrt(3.*delta*gamma/bbb) # gradient energy coefficient  "a"[(J/m)^(1/2)]
www   = 6.*gamma*bbb/delta # potential height W [J/m3]
pmobi = amobi*np.sqrt(2.*www)/(6.*aaa) # mobility of phase-field [m3/(Js)]

In [ ]:
dt = dx*dx/(5.*pmobi*aaa*aaa)/2 # time increment for a time step [s]
nsteps = 1000 # total number of time step
print(dt)

In [ ]:
p  = np.zeros((nx,ny)) # phase-field variable
p_new  = np.zeros((nx,ny)) 

In [ ]:
r_nuclei = 5.*dx # radius of the initial B phase
for i in range(nx):
    for j in range(ny):
        r = np.sqrt( (i *dx)**2 +(j*dy)**2 ) - r_nuclei
        p[i,j] = 0.5*(1.-np.tanh(np.sqrt(2.*www)/(2.*aaa)*r))

In [ ]:
def solve_allen_cahn(p, p_new):
    p_new[1:-1, 1:-1] = p[1:-1, 1:-1] + pmobi * ( 4.*www*p[1:-1, 1:-1]*(1.-p[1:-1, 1:-1])*(p[1:-1, 1:-1]-0.5+3./(2.*www)*eee) \
                                   + aaa*aaa*( (p[2:, 1:-1] - 2*p[1:-1, 1:-1] + p[:-2, 1:-1])/dx/dx \
                                               +(p[1:-1, 2:] - 2*p[1:-1, 1:-1] + p[1:-1, :-2])/dy/dy) ) * dt
    p_new[0,:]    = p_new[1,:]
    p_new[nx-1,:] = p_new[nx-2,:]
    p_new[:,0]    = p_new[:,1]
    p_new[:,ny-1] = p_new[:,ny-2]    
    p[:,:] = p_new[:,:]

In [ ]:
for t in range(nsteps+1):
    solve_allen_cahn(p, p_new)
    
    if t % 100 == 0:
      print('nstep = ', t)
      plt.imshow(p, cmap='bwr')
      plt.title('phase-field')
      plt.colorbar()
      plt.show() 